# Working with Enumerations

##  What is Enumeration?

An enumeration is a set of symbolic names (members) bound to unique, constant values. Within an enumeration, the members can be compared by identity, and the enumeration itself can be iterated over.

- `enumeration` was introduced in Python **3.4**
- It helps you deal with constants in a programmatic manner

## The `enum` Module

The module defines four enumeration classes:

   1. Enum - base class for creating enumerated constants
   2. IntEnum - base class for creating `int` based enums
   3. Flag - base class for creating enum constants that can be combined using the bitwise operations without losing their Flag membership
   4. IntFlag - base class for creating enumerated constants that can be combined using the bitwise operators without losing their IntFlag membership 

    + contains a decorator: `unique`
    + one helper, `auto`.

### Creating a Simple `enum`

- inherit from the base class
- start defining values



In [2]:
import enum
class FirstEnum(enum.Enum):
    CAT = 1
    DOG = 2

In [3]:
FirstEnum.CAT

<FirstEnum.CAT: 1>

### Autogenerating Values

- before the class defined the enum values, but if you do not care, you can automatically generate the values using `auto`


In [4]:
from enum import Enum, auto
class SecondEnum(Enum):
    CAT = auto()
    DOG = auto()

In [5]:
SecondEnum.DOG

<SecondEnum.DOG: 2>

Notice that `SecondEnum.DOG` was given the value of **2** automatically

### Modifying the Autogenerator Values

**Scenerio**: Need the `auto` to assign the value as the `enum` value.


In [7]:
from enum import Enum, auto
class AutoEnum(Enum):
    def _generate_next_value_(name, start, count, last_values):
        return name

class ThirdEnum(AutoEnum):
    CAT = auto()
    DOG = auto()

In [8]:
ThirdEnum.CAT

<ThirdEnum.CAT: 'CAT'>

Now the `CAT` enum value is a string named `CAT`.

## Improving ArcPy with Enums

- `ArcPy` is made up of thousands of methods and classes
- Each input has a set of constants that require strings to be given

#### Scenario: Create a enum class to assist with ArcPy's analysis 

In [29]:
from arcpy.analysis import Buffer, Erase
from arcpy import env
import arcpy, os

In [20]:
arcpy.analysis.Erase?

In [16]:
class ArcPyLineSide(Enum):
    FULL = "FULL"
    LEFT = "LEFT"
    RIGHT = "RIGHT"
    OUTSIDE = "OUTSIDE_ONLY"
class ArcPyDissolve(Enum):
    NONE = "NONE"
    ALL = "ALL"
    LIST = "LIST"


In [30]:
def myworkflow(buffer_fc:str,
               erased_fc:str,
               out_fc:str=None,
               distance:float=10,
               unit:str="meters",
               line_side:ArcPyLineSide=ArcPyLineSide.FULL,
               dissolve:ArcPyDissolve=ArcPyDissolve.ALL):
    """This is a workflow that buffers then removes an area based on geometry.
    
    :retrns: String (output feature class)
    """
    buffer_fc = Buffer(
                        in_features=buffer_fc,
                        out_feature_class=os.path.join("memory", "buffered"),
                        buffer_distance_or_field=f"{distance} {unit}",
                        line_side=line_side.value,
                        line_end_type=None,
                        dissolve_option=dissolve.value,
                        dissolve_field=None,
                        method=None
    )[0]
    erase_fc = Erase(
                    in_features=erased_fc,
                    erase_features=buffer_fc,
                    out_feature_class=out_fc)[0]
    return erase_fc

In [31]:
myworkflow(buffer_fc=r"./data/Maryland_Restaurants_2019.shp", erased_fc="./data/districts.shp")

'./data\\districts_Erase.shp'

### Wrap Up

<img src="./img/enum-all-the-strings.jpg" />


1. Programmatically Figure out Constants
2. Ensures constants are correct
